In [1]:
from aux_fun import get_tokens_inst_iter_folders, get_tokens_inst_iter_df_rg
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import guitarpro as pygp
import glob

path_to_general_read_folder = "..\..\data\DadaGP\DadaGP-v1.1"
path_to_general_write_folder = "..\..\data\BGTG\BGTG-v0.1"

### Arbitrary file generator

This first version outputs the tokens of the selected instruments

In [2]:
# Select the instruments to keep by setting the value to True
instrument_to_keep = {
        "distorted0": False,
        "distorted1": False,
        "distorted2": False,
        "clean0": False,
        "clean1": False,
        "clean2": False,
        "bass": False,
        "leads": False,    
        "pads": False,
        "drums": False,
    }

# get_tokens_inst_iter_folders(instrument_to_keep, path_to_general_read_folder, path_to_general_write_folder)

# 8min10s to generate all the token files with only bass and clean0

# 5min30s to generate all the bass tokens without checking if there is bass or not
# 4min49s for bass tokens if we check in the whole file if there is bass or not
# 6min37s to generate the bass tokens when checking at track-level the presence of bass (11 230 tracks without bass)

# 8min12s to generate the token files with the guitars (clean and distorted)

### Automatic computation of the rythmic instrument's tokens (except bass)

The goal is to iterate over the paths in df_rg_3, and select the tokens of the instruments that are track rythmic.
Generally there should be only one track_rythmic instrument per song, but there can be several if they had the same proportion of rythmic measures over the track.

In [3]:
df_rg_3 = pd.read_csv('../../data/df_rg_3.csv')

In [4]:
# Remove measure level
df_rg_3_track_level = df_rg_3.drop_duplicates(subset=['Dadagp_Name', 'Dadagp_Path', 'File_Name'])

In [5]:
print(df_rg_3_track_level.shape) # One row for each part of each track

df_rg_3_track_level.head()

df_rg_3_track_level[df_rg_3_track_level['is_track_rythmic']]

(54613, 12)


,Fichier,Dadagp_Path,File_Name,Track_Name,Partie,Instrument,Mesure,rg-estimation,Dadagp_Name,is_rythmic,prop_rythmic,is_track_rythmic
0,../data/DadaGP8-gpif/ - Isolated.gpif,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Iso...,- Isolated,Isolated\nDr. Sin,E. Ardanuy,distorted,1,2.570728e-07,distorted0,True,0.480916,True
131,../data/DadaGP8-gpif/ - Tudo Que Vai-Trilha 1....,..\..\data\DadaGP\DadaGP-v1.1\U\Unknown\ - Tud...,- Tudo Que Vai,Tudo Que Vai\nCapital Inicial by Bifaum,Trilha 1,clean,2,3.238349e-05,clean0,True,0.981308,True
386,../data/DadaGP8-gpif/1 God - Grace.gpif,..\..\data\DadaGP\DadaGP-v1.1\1\1 God\1 God - ...,1 God - Grace,Grace\n1 God,Track 1,clean,1,2.090054e-08,clean0,True,1.000000,True
485,../data/DadaGP8-gpif/1 God - Loom.gpif,..\..\data\DadaGP\DadaGP-v1.1\1\1 God\1 God - ...,1 God - Loom,Loom \n1 God,Track 1,clean,1,1.952298e-07,clean0,True,1.000000,True
581,../data/DadaGP8-gpif/1 God - Sativa Arabica.gpif,..\..\data\DadaGP\DadaGP-v1.1\1\1 God\1 God - ...,1 God - Sativa Arabica,sativa arabica\n1 God,Track 1,clean,1,5.664590e-06,clean0,True,1.000000,True
...,...,...,...,...,...,...,...,...,...,...,...,...
4025421,"../data/DadaGP8-gpif/vidal, Gonzalo - Push Up-...","..\..\data\DadaGP\DadaGP-v1.1\V\Vidal, Gonzalo...","vidal, Gonzalo - Push Up",push up\nGonzalo Vidal,Pista Emilio,distorted,2,3.030288e-07,distorted1,True,0.115607,True
4025678,"../data/DadaGP8-gpif/xvwei, xv - City I Have M...","..\..\data\DadaGP\DadaGP-v1.1\X\xvwei, xv\xvwe...","xvwei, xv - City I Have Missing Now",ÎÒË¼ÄîµÄ³ÇÊÐ¡¤ÐíÎ¡ www.xiermusic.com Áø\nNone,°é×à¼ªËû1,clean,1,1.306744e-05,clean0,True,1.000000,True
4025803,"../data/DadaGP8-gpif/xvwei, xv - Lover-°é×à.gpif","..\..\data\DadaGP\DadaGP-v1.1\X\xvwei, xv\xvwe...","xvwei, xv - Lover",lover(ÇéÈË)\nxvwei(ÐíÎº),°é×à,clean,1,1.196826e-03,clean0,True,1.000000,True
4026101,"../data/DadaGP8-gpif/xvwei, xv - Moment Of Tru...","..\..\data\DadaGP\DadaGP-v1.1\X\xvwei, xv\xvwe...","xvwei, xv - Moment Of Truth",Moment of truth\nXu wei,Rhythm guitar,clean,1,3.683445e-11,clean0,True,1.000000,True


In [6]:
get_tokens_inst_iter_df_rg(df_rg_3_track_level, path_to_general_read_folder, path_to_general_write_folder)

  0%|          | 0/23123 [00:00<?, ?it/s]


FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\data\\DadaGP\\DadaGP-v1.1\\U\\Unknown\\ - Isolated.gp4.pygp.txt'